# 

Notebook created by Amol on 26/07/2022
----------------------------------------------------------
File maintained by Amol
---------------------------------------------------------


Seperate code portions can be found in the VS code folder. 
This notebook gives a cohesive environment for the coder to directly manipulate and see the results without worrying too much
about the inside structure of the various functions. 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
#*** means that it is important to run this block of code #
url = 'https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/owid-covid-data.csv'
df = pd.read_csv(url)

#replace value of the path variable with the file path of the csvData.csv file in your own system
path = #"C:/Users/ABHA RANI/Desktop/Amol/Jupyter/RawProjectStorageFile/csvData.csv"
population = pd.read_csv(path)

The below super class contains functions that clean the raw dataset which we are downloading from the Github repo

In [ ]:
#***#
class Super_Normalization():
    # Declaring static url of data
    
    # Declare the path of the population data set
    
    # Declaring standard colors for all countries. These colors will be fixed across all plots
    COLOR = {'France': '#44a2c7', 'Germany': '#f42fa2', 'Finland': '#3b2b99', 'Russia': '#f41f09', 'United Kingdom': '#a25ee1', 'Italy': '#91190a', 'Spain': '#0258bb', 'Sweden': '#2f65bf', 'Slovenia': '#d2ba0d', 'Denmark': '#a83e40', 'Estonia': '#8d3b08', 'Belgium': '#9dbeaa', 'Greece': '#5e4b98', 'Luxembourg': '#b51c57', 'Norway': '#1e5c3e', 'Switzerland': '#2106f2', 'Albania': '#53ace0', 'Austria': '#223406', 'Croatia': '#4f5026', 'Latvia': '#c923cc', 'Romania': '#ae7a50', 'North Macedonia': '#59a61f', 'Serbia': '#96510b', 'Netherlands': '#0525ba', 'Belarus': '#bc6309', 'Iceland': '#e2d4b3',
             'Monaco': '#c28cc5', 'Ireland': '#62b93e', 'San Marino': '#668d02', 'Czechia': '#bcaab2', 'Portugal': '#4851de', 'Andorra': '#c6d06f', 'Ukraine': '#5a7c84', 'Hungary': '#1d9d53', 'Liechtenstein': '#b3a9c0', 'Faeroe Islands': '#d2eef6', 'Poland': '#a55d32', 'Gibraltar': '#bfa438', 'Bosnia and Herzegovina': '#e3cad7', 'Malta': '#2346a6', 'Slovakia': '#732bc8', 'Vatican': '#e52e53', 'Moldova': '#998396', 'Cyprus': '#400089', 'Bulgaria': '#fbe7a8', 'Kosovo': '#f2c023', 'Montenegro': '#a2c1bd', 'Lithuania': '#18a424', 'Isle of Man': '#4a1793', 'Guernsey': '#171d71', 'Jersey': '#256586'}

    def __init__(self, catagory):
        self.data_frame = df
        self.group_by_country = ()
        self.catagory = catagory
        self.test_df = pd.DataFrame()
        
        

    def Filter_Column(self):
        filt = self.data_frame.continent == "Europe"
        self.data_frame = self.data_frame.loc[filt]
        self.data_frame.set_index("date", inplace=True)
        self.data_frame.rename(columns={"location": "country"}, inplace=True)
        self.data_frame = self.data_frame[[
            "country", "new_cases", "new_deaths", "hosp_patients", "icu_patients"]]

    def Grouping_by_country(self):
        self.group_by_country = self.data_frame.groupby("country")

        
    # storing the list of countries with whole column as empty values for a specific parameter
    def list_of_Nan_countries(self, parameter):

        array_of_null_countries = []

        for grouped_country_name, grouped_country_database in self.group_by_country:
            if (len(grouped_country_database) == grouped_country_database[parameter].isnull().sum()):
                array_of_null_countries.append(grouped_country_name)
        return array_of_null_countries

    
    # dataframe of countries for a particular parameter in the form of Pivot Table
    def get_country_df_for_particular_parameter(self, parameter):
        country_df_particular_parameter = self.data_frame.pivot(
            index="date", columns="country", values = parameter)
        return country_df_particular_parameter

    
    # remove the columns of the list of Nan countries from the dataframe
    def delete_Nan_countries_from_df(self, array_of_null_countries, country_df_particular_parameter):

        for df_country in country_df_particular_parameter.columns:
            for array_country in array_of_null_countries:
                if array_country in df_country:
                    del country_df_particular_parameter[df_country]
        return country_df_particular_parameter

    
    # storing the name of the parameters from the data frame of a list
    def getparameter_array(self):
        col_array = []
        for col_name in self.data_frame.columns:
            if(col_name != "date" and col_name != "country"):
                col_array.append(col_name)
        return col_array

    
    # returns the final clean dataset for each parameter in the form of dictionary
    # where the parameter are the keys and data frames are the values
    def get_final_df_Dictionary(self, rolling_days = 14):
        self.Filter_Column()
        self.data_frame.reset_index(inplace=True)
        self.Grouping_by_country()
        dictionary = {}
        col_array = self.getparameter_array()
        for parameter in col_array:
            if parameter == self.catagory:
                Nan_ans = self.list_of_Nan_countries(parameter)
                df_ans = self.get_country_df_for_particular_parameter(parameter)
                deleted_nan_country_df = self.delete_Nan_countries_from_df(
                    Nan_ans, df_ans)
                # final missing values "inside" the dataframe are filled using linear interpolation method
                interpolated_df = deleted_nan_country_df.interpolate(
                    limit_area="inside")
                
                self.test_df = interpolated_df
                print(interpolated_df)
                
                
                #droping specific speical columns whoes data is not in the population database
                del_Col_list = ['Kosovo', 'Jersey', 'Guernsey']
                
                for country in del_Col_list:
                    if country in interpolated_df.columns:
                        interpolated_df = interpolated_df.drop([country], axis = 1)
                        
                    
                 # dividing by the population
                divided_by_population_df = self.Dividing_by_population(interpolated_df)
                
                print(divided_by_population_df)
                
                #finding the rolling _average for better visualisation
                rolling_avg_df = self.rolling_average(divided_by_population_df, rolling_days)
              
                dictionary[parameter] = rolling_avg_df 
                break

        return dictionary
    
    
    def Dividing_by_population(self, interpolated_df):
        
        for index, row in population.iterrows():
            country_name_in_population_df  = row["name"]
            population_value = row["pop"]
            if country_name_in_population_df in interpolated_df.columns:
                interpolated_df.loc[:, country_name_in_population_df] = interpolated_df.loc[:, country_name_in_population_df].apply(lambda x: x/population_value)
                
        return interpolated_df
            

    # The Type argument is the type of Normalization
    # The catagory argument is a catagory such as 'new_cases'
    # The countries = None arguments plots all countries if there are no specified countries
    def plot_data_frame(self, DataFrame, Type, catagory, countries=None, rolling_days = 14, free_pass = False):
        
        #DataFrame = self.rolling_average(DataFrame, rolling_days)
        if not free_pass:
            DataFrame.reset_index(inplace=True)
            DataFrame['date'] = pd.to_datetime(DataFrame['date'])
        

        if countries != None :
            for column in countries:
                if column == 'date':
                    continue
                if column in self.test_df.columns:
                    plt.plot(
                    DataFrame.date, DataFrame[column], color=Super_Normalization.COLOR[column], label=column )
                #plt.xticks(DataFrame.date[::100])
        if countries == None:
            for column in DataFrame:
                if column == 'date':
                    continue
                if column in self.test_df.columns:
                    plt.plot(DataFrame['date'], DataFrame[column], color=Super_Normalization.COLOR[column], label=column)
                #plt.xticks(DataFrame.date[::100])
        plt.title("Normalizing each country with " +
                  Type + " Maximum " + catagory + "\n" + "Frame Size 120")
        plt.xlabel("Dates")
        plt.ylabel("Normalized to 1")
        plt.xticks(DataFrame.date[::90])
        plt.tick_params(axis='x', labelrotation=90)
        #plt.legend(bbox_to_anchor=(1, 1), loc='upper left')
        #plt.legend()
        plt.savefig("Raaju1234567" +'.png', dpi = 300)
        plt.show()
        return DataFrame
        
    
    # new_df = dataframe with date as index and countries as column
    # rolling_days = number of days you want to take average of (strongly recommended that the value should be a multiple of 7)
    def rolling_average(self, new_df, rolling_days):
        
        row_count = new_df.shape[0]
        column_count = new_df.shape[1]

        temp_data_frame = new_df.copy()

        for country_index in range(column_count):

            index_counter = 0

            # for each date in each country
            for date_index in range(row_count):

                # surpass all the Nan values in the dataframe then only proceeding
                if not (np.isnan(new_df.iloc[date_index, country_index])):

                    # summing of the rolling days from the copied data frame
                    # assuming that the dataframe is larger than the rolling_days parameter
                    rolling_counter_index = date_index
                    ending_rolling_counter_index = date_index + rolling_days

                    # finding the mean of all the days within the rolling days window
                    rolling_days_mean = temp_data_frame.iloc[rolling_counter_index:
                                                             ending_rolling_counter_index, country_index].mean()       
                    
                    # updating the new value in the original dataframe
                    new_df.iloc[ending_rolling_counter_index - 1, country_index] = rolling_days_mean

                    # reached the end of the dataframe
                    if(ending_rolling_counter_index - 1 == row_count-1):
                        break

                    # this function will work for the first number of rolling days 
                    # except the last day where we are actually filling the new average value
                    if index_counter < rolling_days -1:
                        # removing (here filling with Nan value) the first rolling days values from the dataframe
                        new_df.iloc[date_index, country_index] = np.nan
                        index_counter += 1
        return new_df 


 ## Normalisation Techniques
 

You can run any one block of code
1. Global Static

In [ ]:
class Global_Static(Super_Normalization):
    # Initiailizig super class and this class
    # also assigning a catagory to each instant. Such as 'new_cases'
    def __init__(self, catagory):
        Super_Normalization.__init__(self, catagory)
        self.catagory = catagory
        self.Dataframe_with_countries_as_column = Super_Normalization.get_final_df_Dictionary(self)[
            self.catagory]

    # Function divides each data point by the global maximum
    # uses the applymap method which acts on each data point in the data set
    def Divide_by_global_max(self):
        global_max = self.Dataframe_with_countries_as_column.max().max()
        self.Dataframe_with_countries_as_column = self.Dataframe_with_countries_as_column.applymap(
            lambda x: x/global_max)
        
    # Function plots the new cases from specified country normalized to the global maximum
    def plot_data_frame(self, countries=None):
        self.Dataframe_with_countries_as_column = super().plot_data_frame(
            self.Dataframe_with_countries_as_column, "Global Static", self.catagory, countries)


2. Global Dynamic Max

In [ ]:

class Global_Dynamic(Super_Normalization):
    # Initiailizing super class and this class
    # also assigning a catagory to each instant. Such as 'new_cases'
    def __init__(self, catagory):
        Super_Normalization.__init__(self, catagory)
        self.catagory = catagory
        self.Dataframe_with_countries_as_column = Super_Normalization.get_final_df_Dictionary(self)[
            self.catagory]
        #creating a column of matrix of size (row size * 1)
        self.max_array = [0]*self.data_frame.shape[0]

    # Creates the maximum array
    # The maximum array holds global maximum value for all the frames that a particular 
    # data point was under corresponding to a particular date (row) and country (column) 
    def Create_max_array(self, frame_size):
        
        row_size = self.Dataframe_with_countries_as_column.shape[0]
        
        for i in range(0, row_size):
            
            # condition for the last frame
            if i > row_size - frame_size:
                
                break
                
                
            
            #getting the maximum value
            #there should not be +1 in frame size as we looking for 90 days worth of dataset not 91
            global_frame_max = self.Dataframe_with_countries_as_column.iloc[i: i + frame_size+1].max(
            ).max()
            
            #updating the matrix with the global max value for that window if that value is the greatest of all
            for j in range(i, i + frame_size+1):
                if self.max_array[j] < global_frame_max:
                    self.max_array[j] = global_frame_max

    # Function divides each data point by the global maximum
    def Divide_by_max_array(self):
        row_size = self.Dataframe_with_countries_as_column.shape[0]
        for i in range(0, row_size):
            self.Dataframe_with_countries_as_column.iloc[i] = self.Dataframe_with_countries_as_column.iloc[i]/self.max_array[i]

    # Function plots the new cases from specified country normalized to the global maximum
    def plot_data_frame(self, countries=None):
        super().plot_data_frame(
            self.Dataframe_with_countries_as_column, "Global Dynamic", self.catagory, countries)


3. Global Dynamic Mean

In [ ]:
class Global_Dynamic_Mean(Super_Normalization):
    # Initiailizing super class and this class
    # also assigning a catagory to each instant. Such as 'new_cases'
    def __init__(self, catagory):
        Super_Normalization.__init__(self, catagory)
        self.catagory = catagory
        self.Dataframe_with_countries_as_column = Super_Normalization.get_final_df_Dictionary(self)[
            self.catagory]

    # Function divides and adds each data points of frame by the global maximum
    # uses the applymap method which acts on each data point in the data set

    def Divide_by_max_and_add(self, frame_size):
        row_size = self.Dataframe_with_countries_as_column.shape[0]
        temp_data_frame = self.Dataframe_with_countries_as_column.copy()
        
        # setting each element of the data frame with the value of 0
        self.Dataframe_with_countries_as_column = self.Dataframe_with_countries_as_column.applymap(lambda x: 0)
        
        for i in range(1, row_size):
            
            # condition for the last frame
            if i > row_size-frame_size:
                global_dynamic_max = temp_data_frame.iloc[i:row_size].max(
                ).max()
                self.Dataframe_with_countries_as_column.iloc[i:row_size] += temp_data_frame[i:row_size].applymap(
                    lambda x: x/global_dynamic_max)
                continue
               
            # finding the global maximum within that frame
            global_dynamic_max = temp_data_frame.iloc[i:i+ frame_size+1].max().max()
            
            # the apply function acts on each country's column's data point in the dataframe
            self.Dataframe_with_countries_as_column.iloc[i:i+frame_size + 1] += temp_data_frame.iloc[i:i+frame_size+1].applymap(lambda x: x/global_dynamic_max)

    # Function takes the mean of each data point according to the number of times vaules have been added to it
    def Divide_by_frame_size(self, frame_size):
        row_size = self.Dataframe_with_countries_as_column.shape[0]
        count_foward = 1
        for i in range(0, row_size):
            
            # Condition for the data points within the first frame
            # ??????Shouldn't the first frame columns be also dropped
            if i < frame_size:
                
                # dividing all the countries for the same date with the frame size
                self.Dataframe_with_countries_as_column.iloc[i] = self.Dataframe_with_countries_as_column.iloc[i]/count_foward
                count_foward += 1
                continue
                
            self.Dataframe_with_countries_as_column.iloc[i] = self.Dataframe_with_countries_as_column.iloc[i]/frame_size
            
        self.Dataframe_with_countries_as_column.drop(self.Dataframe_with_countries_as_column.index[row_size-(frame_size+1):row_size], inplace=True)

        
    # Function plots the new cases from each country normalized to the global maximum
    def plot_data_frame(self, countries=None):
        super().plot_data_frame(
            self.Dataframe_with_countries_as_column, "Global Dynamic Mean", self.catagory, countries)



**Main Running Script**

In [ ]:
#***#

frame_size = 120
i = 0

'''These are the list of countries that we are working with''' 
List = ['Russia',
'Germany','United Kingdom','France','Italy','Spain','Ukraine','Poland','Romania','Netherlands','Belgium','Czechia','Greece','Sweden','Portugal','Hungary',
'Belarus','Austria','Switzerland','Serbia','Bulgaria','Denmark','Finland','Norway','Slovakia','Ireland','Croatia','Moldova','Bosnia and Herzegovina','Albania','Lithuania','North Macedonia','Slovenia','Latvia','Estonia',
'Cyprus','Luxembourg','Montenegro','Malta','Iceland','Isle of Man','Andorra','Faeroe Islands','Monaco','Liechtenstein','San Marino','Gibraltar',
'Vatican']

"""
Global Static Technique

To run it just uncomment the below lines of code

"""
# Catagory = ["new_cases", "new_deaths", "hosp_patients", "icu_patients"]
# Ldm = Global_Static(Catagory[i])
# Ldm.Divide_by_global_max()
# Ldm.plot_data_frame()

"""
Global Dynamic Max

To run it just uncomment the below lines of code

"""

# Catagory = ["new_cases", "new_deaths", "hosp_patients", "icu_patients"]
# country = ["France", "Germany", "Italy"]
# Ldm = Global_Dynamic(Catagory[i])
# Ldm.Create_max_array(90)
# Ldm.Divide_by_max_array()
# Ldm.plot_data_frame()


"""
Global Dynamic Mean

To run it just uncomment the below lines of code

"""
# Catagory = ["new_cases", "new_deaths", "hosp_patients", "icu_patients"]
# Ldm = Global_Dynamic_Mean(Catagory[i])
# Ldm.Divide_by_max_and_add(frame_size)
# Ldm.Divide_by_frame_size(frame_size)
# Ldm.plot_data_frame()

*to see the plot of a specific country*

In [ ]:
Ldm.plot_data_frame(["United Kingdom"])

## Data Analysis

(Probability Transition Matrix)

**Below functions will help you in generation of probability transition matrix through different techniques**

In [ ]:


def sliding_technique_matrix_formation (frame_size, data_frame, country_list):
    
    
    #deep copying the orginal data frame
    signed_dataframe = data_frame.copy()
    
    for country in country_list:
        
        #intialising variables
        start_index = 0
        end_index = 0
        sliding_value_of_each_frame = 7 #days
        country_val_series = data_frame[country]
        length_of_the_series = country_val_series.shape[0]
        
        #finding the number of leftover days: sure to be less than 7 days
        remaining_unanalysed_days = (length_of_the_series - frame_size) % sliding_value_of_each_frame

        for slide in range(0,length_of_the_series, sliding_value_of_each_frame):
            
            start_index = slide
            end_index = slide + frame_size
            
            if end_index > length_of_the_series:
                break
            
            #finding the min and max value from the country's column for that particular frame
            min_val_in_series = country_val_series.iloc[start_index : end_index].min()
            max_val_in_series = country_val_series.iloc[start_index : end_index].max()
            
            #finding the threshold values for dividing that frame's country's values into three groups. 
            bins = np.linspace(min_val_in_series, max_val_in_series, 4)
            group_names = ["-","+","++"]
            
            #updating the frame in the new signed data_frame with the group names using the cut function from pandas
            signed_dataframe[country].iloc[start_index : end_index] = pd.cut(data_frame[country].iloc[start_index : end_index], bins, labels = group_names, include_lowest = True)
            # Intervals are like this [a,b],(b,c], (c,d] for -, + and ++
            
            print("Country Name: ", country)
            print("From data: ", data_frame["date"][start_index] , " to ", data_frame["date"][end_index - 1])
            probability_transition_matrix_generator(signed_dataframe[country][start_index : end_index], start_index, end_index)
            
def probability_transition_matrix_generator(country_series_values, start_index, end_index):
    
    #creating a numpy matrix
    transition_matrix = np.matrix([[0.0,0,0],
                                   [0,0,0],
                                   [0,0,0]])
    
    
    #iterating and comparing each value of the series
    #- 1 because in the call the country's index on the right hand side in the parameter is exclusive
    for index in range(start_index, end_index - 1):

        if country_series_values[index] == "-" and country_series_values[index+1] == "-":
            transition_matrix[0,0] += 1
            
        elif country_series_values[index] == "-" and country_series_values[index+1] == "+":
            transition_matrix[0,1] += 1  

        elif country_series_values[index] == "-" and country_series_values[index+1] == "++":
            transition_matrix[0,2] += 1

        elif country_series_values[index] == "+" and country_series_values[index+1] == "-":
            transition_matrix[1,0] += 1

        elif country_series_values[index] == "+" and country_series_values[index+1] == "+":
            transition_matrix[1,1] += 1

        elif country_series_values[index] == "+" and country_series_values[index + 1] == "++":
            transition_matrix[1,2] += 1

        elif country_series_values[index] == "++" and country_series_values[index + 1] == "-":
            transition_matrix[2,0] += 1

        elif country_series_values[index] == "++" and country_series_values[index + 1] == "+":
            transition_matrix[2,1] += 1

        elif country_series_values[index] == "++" and country_series_values[index + 1] == "++":
            transition_matrix[2,2] += 1 

    sum_of_columns = transition_matrix.sum(axis = 1)
    transition_matrix[0,:] = transition_matrix[0,:]/sum_of_columns[0,0]
    transition_matrix[1,:] = transition_matrix[1,:]/sum_of_columns[1,0]
    transition_matrix[2,:] = transition_matrix[2,:]/sum_of_columns[2,0]

    print("Transition Matrix: ", "\n\n", transition_matrix, "\n")

def customised_probability_transition_matrix_generator(data_frame, country_list ):
    
    for country in country_list:
        val_series = data_frame[country]
        start_index = 0
        end_index = data_frame.shape[0] - 1
        print(country, " \n")
        probability_transition_matrix_generator(val_series, start_index, end_index)
        
    
def creating_signed_dataframe(data_frame):
    
    signed_data_frame = data_frame.copy()
    
    for country in data_frame.columns:
        if country == "date": continue
            
        country_column = data_frame[country]
        min_val = country_column.min()
        max_val = country_column.max()
        
        #tranforming the unsigned dataframe into signed dataframe
        signed_data_frame[country] = assigning_sign_values(min_val, max_val, data_frame, country)
    
    return signed_data_frame

def assigning_sign_values(min_val, max_val, data_frame, country):
    
    #finding the threshold values for dividing that frame's country's values into three groups. 
    bins = np.linspace(min_val, max_val, 4)
    group_names = ["-","+","++"]
            
    #dividing into groups using the cut function from pandas
    return pd.cut(data_frame[country], bins, labels = group_names, include_lowest = True)
    # Intervals for the groups are like-- [a,b],(b,c], (c,d] for -, + and ++

def creating_weightage_data_frame (signed_data_frame):
    
    #doing this so that the date's column does not get converted also
    signed_data_frame.set_index("date", inplace = True)
    weightage_signed_data_frame = signed_data_frame.applymap(lambda x: sign_to_weight_val_converter(x))
    weightage_signed_data_frame.reset_index(inplace = True)
    return weightage_signed_data_frame


def sign_to_weight_val_converter(val):
    
    if val == "-":
        return -1
    elif val == "+":
        return 0
    elif val == "++":
        return 1
    
def creating_weekly_transition_dataframe(average_weeks_in_days, weightage_signed_data_frame):
    
    row = weightage_signed_data_frame.shape[0]
    column = weightage_signed_data_frame.columns
    end_index = math.floor(row/average_weeks_in_days)
    week_counter = 0
    weightage_signed_data_frame['date'] = weightage_signed_data_frame['date'].astype(str)
    
    #creating a new dataframe of size 1/average_weeks_in_days th of the weightage signed data frame
    weekly_transition_dataframe = pd.DataFrame(index = range(0, end_index), columns = column)
    
    
    for col in column:
        
        if col == "date": continue
            
        col_val = weightage_signed_data_frame[col]
        row_size = weightage_signed_data_frame.shape[0]
        
        for x in range(0, row_size, average_weeks_in_days):
            start_index = x
            end_index = x + average_weeks_in_days
            
            if end_index > row_size: break
                
            weekly_transition_dataframe['date'][x/average_weeks_in_days] = weightage_signed_data_frame['date'][end_index - 1]
            
            if any (np.isnan(weightage_signed_data_frame[col][start_index:end_index])):
                 continue
                
            
            val = round(weightage_signed_data_frame[col][start_index:end_index].mean())
            
            if val == -1:
                weekly_transition_dataframe[col][x/average_weeks_in_days] = "-"
            elif val == 0:
                weekly_transition_dataframe[col][x/average_weeks_in_days] = "+"
            elif val == 1:
                weekly_transition_dataframe[col][x/average_weeks_in_days] = "++"
            
    
    weekly_transition_dataframe['date'] = pd.to_datetime(weekly_transition_dataframe['date'])
    return weekly_transition_dataframe
    
    

**Main Script**

In [ ]:
#***#

original_df = Ldm.Dataframe_with_countries_as_column.copy()

#removing empty rows
original_df.drop([0,1], axis=0, inplace=True)
original_df.drop(["index"], axis=1, inplace=True)
original_df.reset_index(drop = True, inplace = True)


"""
Technique 1:
output will give you probability transition matrix of each country using the sliding technique on the complete orignal database
(uncomment the below lines and run to unleash their power)
(you change or add countries in the list)
"""
#sliding_technique_matrix_formation(frame_size = 150, data_frame = original_df , country_list = ['France', 'Germany', "United Kingdom"])

"""
Technique 2:
output will give you probability transition matrix of country using the sliding technique on a weekly average database
(uncomment the below lines once by selecting all the below lines and clicking  ctrl + / and then run to unleash their power)
(you can also add more countries in the list)
"""
# #the output dataframe has been catagorised into three groups of -,+, and ++ 
# signed_dataframe = creating_signed_dataframe(original_df)
# #the output dataframe has -1,0,1 instead of -,+,++
# weightage_signed_df = creating_weightage_data_frame(signed_dataframe)

# #the output dataframe will take average of contiguous days' values and then store it as an single unit
# #e.g avg of first 7 days and then store the mean value as week 1, next 7 days and storing it as week 2
# weekly_transition_df = creating_weekly_transition_dataframe(average_weeks_in_days = 10, weightage_signed_data_frame = weightage_signed_df)
# print(weekly_transition_df["Germany"][0:150])
# #display the probability transition matrix of the countries mentioned in the list
# customised_probability_transition_matrix_generator(weekly_transition_df, country_list = ["France","Germany", "United Kingdom"])
